In [1]:
from pathlib import Path

import torch
import numpy as np
import pandas as pd
import hvplot.pandas

import diffhydro as dh
import diffhydro.pipelines as dhp
import xtensor as xt

from utils.io import load_ono_data, split_and_normalize_data

### Parameters

In [2]:
N = 10                  # Repeat experiment 10 times
lr = .005               # Initial Runoff Learning rate
wd = .001               # LSTM Weight decay

n_epoch = 100           # training epoch
n_iter  = 20            # batch per epoch

init_window = 100       # LSTM init window
pred_len = 200

device = "cuda:6"       #  Device: Assume available to NVIDIA GPU
root = Path("./data")

### Load data

In [3]:
inp, lbl, statics, g = load_ono_data(root)
inp_tr, inp_te, lbl_tr, lbl_te, lbl_std = split_and_normalize_data(inp, lbl)

In [4]:
tr_ds  = dhp.BaseDataset(x=inp_tr, y=lbl_tr, g=g, 
                         init_len=init_window, 
                         pred_len=pred_len, 
                         statics=statics)
val_ds = dhp.BaseDataset(x=inp_te, y=lbl_te, g=g, 
                         init_len=init_window, 
                         pred_len=pred_len, 
                         statics=statics) 

param_model = dhp.MLP(2,2)
model  = dhp.RunoffRoutingModel(param_model, dt=.25, max_delay=100).to(device)
module = dhp.RunoffRoutingModule(model, tr_ds, val_ds,
                                 device=device,
                                 batch_size=256,
                                 clip_grad_norm=1,
                                 routing_lr=10**-4, 
                                 routing_wd=10**-3, 
                                 runoff_lr=.005, 
                                 runoff_wd=.001,
                                 scheduler_step_size=None,
                                 scheduler_gamma=.3)

In [ ]:
trloss, valloss = module.train(n_epoch=n_epoch, n_iter=n_iter)

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [ ]:
(trloss.hvplot() * valloss.hvplot()).opts(ylim=(.8, 1))

In [ ]:
y_te, o_te = module.extract_test(device)